# Introduction to Neural Network

The widespread adoption of artificial intelligence in recent years has been largely driven by advancement in neural networks. Neural networks is behind systems ranging from 
<a href="https://deepmind.com/research/alphago/">AlphaGo</a>, 
<a href="https://translate.google.com/">Google Translate</a> 
to <a href="https://www.tesla.com/en_HK/autopilot">Tesla Autopilot</a>.

Neural network is fundamentally numeric computation, so any software with decent numeric computation capabilities can be used to construct and train a neural network. That said, while in theory you can construct a neural network in Excel, in practice it will be very troublesome since Excel is not designed with neural network in mind. Libraries are that specifically geared toward neural network include:
- Google's <a href="https://www.tensorflow.org/">Tensorflow</a>
- Microsoft's <a href="https://github.com/Microsoft/CNTK">CNTK</a>
- Facebook's <a href="http://pytorch.org/">PyTorch</a> and <a href="https://caffe2.ai/">Caffe2</a>
- Intel's <a href="https://ai.intel.com/neon/">neon</a>
- <a href="http://deeplearning.net/software/theano/">Theano</a> and <a href="http://caffe.berkeleyvision.org/">Caffe</a>

In this course we will focus on using <a href="https://keras.io/">```keras```</a>, which is a high-level library for constructing neural networks. Keras runs on top of a numerical computation library of your choice, defaulting to ```tensorflow```. A library such as Keras significantly simplify the workflow of constructing and training neural networks. 

<img src="http://www.ticoneva.com/econ/econ4130/images/nn_libraries.png" width="80%">

Before we start, we will first disable the server's GPU so that everything runs on its CPU. Later we will turn it back on to see how much speed up we can get.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

## A Simple Example: Binary Neural Network Classifier

As a first example, we will train a neural network to the following classification task:

|y|x1|x2|
|-|-|-|
|0|1|0|
|1|0|1|

To be clear: there is absolutely no need to use neural network for such as simple task. A simpler model will train a lot faster and potentially with better accuracy.

We first generate the data:

In [2]:
import numpy as np 
from sklearn.model_selection import train_test_split

#Generate 2000 samples. [1,0] -> 0, [0,1] -> 1
X = np.repeat([[1,0]], 1000, axis=0)
y = np.repeat([0], 1000, axis=0)
X = np.append(X,np.repeat([[0,1]], 1000, axis=0),axis=0)
y = np.append(y,np.repeat([1], 1000, axis=0),axis=0)

#Shuffle and split data into train set and test set
X_train, X_test, y_train, y_test = train_test_split(X,y)

We will construct a neural network classifier for this task. 

A neural network model is made up of multiple layers. The simpliest model would have three layers:
- An *input layer*. This layer specify the nature of the input data. In this example, we only need to tell Keras that we have two variables to input.
- A *hidden layer*. This layer contains neuron(s) that process the input data.
- An *ouput layer*. The neurons in this layer process the output from the hidden layer and generate predictions. This layer contains as many neurons as the number of target variables we try to predict. 

Below is the simplest neural network one can come up with, with only one hidden neuron. The neuron computes the following function:
}
$$
F \left( b + \sum\nolimits_{i}{w_{i}x_{i}} \right)
$$

where $x_i$ are inputs, b the intercept (called *bias* in machine learning), $w_i$ coefficients (called *weights*) and $F$ is an *activation function*. In this example we will use the logistic function (also called the *sigmoid function*) as the activation function:

$$
F(z) = \frac{e^z}{1+e^z}
$$

So the neuron is essentially a logit regression.

In [3]:
from keras.layers import Input, Dense
from keras.models import Model

# Set up layers 
inputs = Input(shape=(2,))
x = Dense(1, activation='sigmoid')(inputs)
predictions = Dense(1, activation='sigmoid')(x)

# Set up model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train,y_train,epochs=50)  # starts training

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Epoch 1/50
1500/1500 [==============================] - 0s 108us/step - loss: 0.8546 - acc: 0.5027
Epoch 2/50
1500/1500 [==============================] - 0s 28us/step - loss: 0.8253 - acc: 0.5027
Epoch 3/50
1500/1500 [==============================] - 0s 31us/step - loss: 0.7997 - acc: 0.5027
Epoch 4/50
1500/1500 [==============================] - 0s 31us/step - loss: 0.7766 - acc: 0.5027
Epoch 5/50
1500/1500 [==============================] - 0s 31us/step - loss: 0.7560 - acc: 0.5027
Epoch 6/50
1500/1500 [==============================] - 0s 28us/step - loss: 0.7378 - acc: 0.5027
Epoch 7/50
1500/1500 [==============================] - 0s 31us/step - loss: 0.7214 - acc: 0.5027
Epoch 8/50
1500/1500 [==============================] - 0s 28us/step - loss: 0.7069 - acc: 0.5027
Epoch 9/50
1500/1500 [==============================] - 0s 29us/step - loss: 0.6938 - acc: 0.5027
Epoch 10/50
1500/1500 [==============================] - 0s 27us/step - loss: 0.6820 - acc: 0.5027
Epoch 11/50
1500/1

Out-of-sample test can be conducted with ```model.evaluate()```:

In [4]:
model.evaluate(x=X_test,y=y_test)

500/500 [==============================] - 0s 59us/step


[0.3190232439041138, 1.0]

The first number is the model's loss while the subsequent numbers are the metrics we specified. In our case, they are ```binary_crossentropy``` and ```accuracy``` respectively.

Unlike OLS, a neural network's performance could vary across runs. Run the code a few more times and see how the performance vary.

Make prediction (this is called *inference* in machine learning) with ```model.predict()```:

In [5]:
x = np.array([[0,1]])
print(model.predict(x))

[[0.6856742]]


## Neural Network Regression

Next we are going use a neural network in a regression task. The true data generating process (DGP) is as follows:

$$
y = x^5 -2x^3 + 6x^2 + 10x - 5
$$

The model does not know the true DGP, so it needs to figure out the relationship between $y$ and $x$ from the data.

First we generate the data:

In [6]:
#Generate 1000 samples
X = np.random.rand(1000,1)
y = X**5 - 2*X**3 + 6*X**2 + 10*X - 5

#Shuffle and split data into train set and test set
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [7]:
inputs = Input(shape=(1,))
x = Dense(100, activation='sigmoid')(inputs)
predictions = Dense(1, activation='linear')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='adam',
              loss='mean_squared_error')
model.fit(X_train,y_train,epochs=200)
model.evaluate(x=X_test,y=y_test)

Epoch 1/200
750/750 [==============================] - 0s 149us/step - loss: 24.7736
Epoch 2/200
750/750 [==============================] - 0s 28us/step - loss: 20.4868
Epoch 3/200
750/750 [==============================] - 0s 26us/step - loss: 19.1489
Epoch 4/200
750/750 [==============================] - 0s 28us/step - loss: 18.7362
Epoch 5/200
750/750 [==============================] - 0s 25us/step - loss: 18.5726
Epoch 6/200
750/750 [==============================] - 0s 28us/step - loss: 18.4260
Epoch 7/200
750/750 [==============================] - 0s 26us/step - loss: 18.2642
Epoch 8/200
750/750 [==============================] - 0s 27us/step - loss: 18.1041
Epoch 9/200
750/750 [==============================] - 0s 26us/step - loss: 17.9185
Epoch 10/200
750/750 [==============================] - 0s 28us/step - loss: 17.7380
Epoch 11/200
750/750 [==============================] - 0s 29us/step - loss: 17.5465
Epoch 12/200
750/750 [==============================] - 0s 27us/step - lo

750/750 [==============================] - 0s 24us/step - loss: 0.2015
Epoch 99/200
750/750 [==============================] - 0s 25us/step - loss: 0.2014
Epoch 100/200
750/750 [==============================] - 0s 25us/step - loss: 0.1993
Epoch 101/200
750/750 [==============================] - 0s 23us/step - loss: 0.2007
Epoch 102/200
750/750 [==============================] - 0s 24us/step - loss: 0.1987
Epoch 103/200
750/750 [==============================] - 0s 27us/step - loss: 0.1977
Epoch 104/200
750/750 [==============================] - 0s 23us/step - loss: 0.1970
Epoch 105/200
750/750 [==============================] - 0s 24us/step - loss: 0.1961
Epoch 106/200
750/750 [==============================] - 0s 24us/step - loss: 0.1958
Epoch 107/200
750/750 [==============================] - 0s 23us/step - loss: 0.1952
Epoch 108/200
750/750 [==============================] - 0s 27us/step - loss: 0.1950
Epoch 109/200
750/750 [==============================] - 0s 24us/step - loss: 0.

750/750 [==============================] - 0s 23us/step - loss: 0.1334
Epoch 195/200
750/750 [==============================] - 0s 25us/step - loss: 0.1325
Epoch 196/200
750/750 [==============================] - 0s 27us/step - loss: 0.1319
Epoch 197/200
750/750 [==============================] - 0s 28us/step - loss: 0.1316
Epoch 198/200
750/750 [==============================] - 0s 26us/step - loss: 0.1317
Epoch 199/200
750/750 [==============================] - 0s 27us/step - loss: 0.1298
Epoch 200/200
250/250 [==============================] - 0s 85us/step


0.12120804953575134

We are going to run the model through different settings. The function contains everything we have coded previously:

In [20]:
import time

def polyNN(data,hidden_count=100,epochs=200,batch_size=32):
    
    #Record the start time
    start = time.time()
    
    #Unpack the data
    X_train, X_test, y_train, y_test = data
    
    #Layers
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(hidden_count, activation='sigmoid')(inputs)
    predictions = Dense(1, activation='linear')(x)

    #Model
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam',
                  loss='mean_squared_error')
    model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size,verbose=0) #Do not display progress
    
    #Collect and display info
    param_count = model.count_params()
    loss = round(model.evaluate(x=X_test,y=y_test,batch_size=batch_size,verbose=0),5)
    elapsed = round(time.time() - start,2)    
    print("Hidden count:",str(hidden_count).ljust(5),
          "Parameters:",str(param_count).ljust(8),
          "loss:",str(loss).ljust(10),
          "Time:",str(elapsed)+"s",
         )

Now we can easily try out different settings:

In [21]:
data = train_test_split(X,y)

polyNN(data,hidden_count=1)
polyNN(data,hidden_count=10)
polyNN(data,hidden_count=50)
polyNN(data,hidden_count=100)
polyNN(data,hidden_count=500)

Hidden count: 1     Parameters: 4        loss: 15.44601   Time: 4.51s
Hidden count: 10    Parameters: 31       loss: 0.09365    Time: 4.52s
Hidden count: 50    Parameters: 151      loss: 0.07055    Time: 4.7s
Hidden count: 100   Parameters: 301      loss: 0.10033    Time: 4.7s
Hidden count: 500   Parameters: 1501     loss: 0.11597    Time: 5.05s


Here we see the universal approximation theory in work: the more neurons we have the better the fit.

One trick that can often improve performance: *standardizing* data.

In [22]:
from sklearn import preprocessing
scalar = preprocessing.StandardScaler().fit(X)
X_std = scalar.transform(X)

data = train_test_split(X_std,y)

polyNN(data,hidden_count=1)
polyNN(data,hidden_count=10)
polyNN(data,hidden_count=50)
polyNN(data,hidden_count=100)
polyNN(data,hidden_count=500)

Hidden count: 1     Parameters: 4        loss: 7.84441    Time: 4.68s
Hidden count: 10    Parameters: 31       loss: 0.1123     Time: 4.73s
Hidden count: 50    Parameters: 151      loss: 0.01967    Time: 4.72s
Hidden count: 100   Parameters: 301      loss: 0.00905    Time: 4.8s
Hidden count: 500   Parameters: 1501     loss: 0.01       Time: 5.35s


## Speed Things Up

Due to its complexity, neural network trains a lot slower than the other techniques we have covered previously. To speed up training, we can ask Keras to go through more samples before updating the model's parameters by specifying a larger ```batch_size```. Doing so allows Keras to make better use of the CPU's parallel processing capabitilies.

Keras' default batch size is 32. We will try 128 instead:

In [23]:
batch_size = 128
polyNN(data,hidden_count=1,batch_size=batch_size)
polyNN(data,hidden_count=10,batch_size=batch_size)
polyNN(data,hidden_count=50,batch_size=batch_size)
polyNN(data,hidden_count=100,batch_size=batch_size)
polyNN(data,hidden_count=500,batch_size=batch_size)

Hidden count: 1     Parameters: 4        loss: 19.49446   Time: 1.77s
Hidden count: 10    Parameters: 31       loss: 1.85994    Time: 1.81s
Hidden count: 50    Parameters: 151      loss: 0.11633    Time: 1.81s
Hidden count: 100   Parameters: 301      loss: 0.07967    Time: 1.95s
Hidden count: 500   Parameters: 1501     loss: 0.10118    Time: 2.73s


## Running Model on GPU

In [16]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

<module 'keras' from '/usr/local/lib/python3.5/dist-packages/keras/__init__.py'>

In [17]:
polyNN(data,hidden_count=1)

Hidden count: 1     Parameters: 4        loss: 6.26372    Time: 4.36s


In [ ]:
batch_size = 1000
polyNN(data,hidden_count=1,batch_size=batch_size)
polyNN(data,hidden_count=10,batch_size=batch_size)
polyNN(data,hidden_count=50,batch_size=batch_size)
polyNN(data,hidden_count=100,batch_size=batch_size)
polyNN(data,hidden_count=500,batch_size=batch_size)

In [ ]:
batch_size = 1000
epochs = 600
polyNN(data,hidden_count=1,epochs=epochs,batch_size=batch_size)
polyNN(data,hidden_count=10,epochs=epochs,batch_size=batch_size)
polyNN(data,hidden_count=50,epochs=epochs,batch_size=batch_size)
polyNN(data,hidden_count=100,epochs=epochs,batch_size=batch_size)
polyNN(data,hidden_count=500,epochs=epochs,batch_size=batch_size)

## Deep Learning

*Deep learning* is the stacking of multiple layers of neurons. Holding the number of parameters constant, this often performs better than having only a single hidden layer.

In [19]:
def polyDNN(data,
            hidden_count=100,
            epochs=200,
            batch_size=1000,
            layers=1,
            activation='sigmoid'
           ):
        
    start = time.time()    

    X_train, X_test, y_train, y_test = data
    
    #Layers
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(hidden_count, activation=activation)(inputs)
    for _ in range(1,layers):
        x = Dense(hidden_count, activation=activation)(x)    
    predictions = Dense(1, activation='linear')(x)

    #Model
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam',
                  loss='mean_squared_error')
    model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size,verbose=0) 
    
    #Collect and display info
    param_count = model.count_params()
    loss = round(model.evaluate(x=X_test,y=y_test,batch_size=batch_size,verbose=0),5)
    elapsed = round(time.time() - start,2)    
    print("Hidden count:",str(hidden_count).ljust(5),
          "Parameters:",str(param_count).ljust(8),
          "loss:",str(loss).ljust(10),
          "Time:",str(elapsed)+"s",
         )

Let us run the model through various settings. I have chosen the neuron count such that the number of parameters is roughly the same as in the single-layer case.

In [21]:
polyDNN(data,hidden_count=1,layers=2,activation='sigmoid')
polyDNN(data,hidden_count=10,layers=2,activation='sigmoid')
polyDNN(data,hidden_count=15,layers=2,activation='sigmoid')
polyDNN(data,hidden_count=36,layers=2,activation='sigmoid')

Hidden count: 1     Parameters: 6        loss: 18.50988   Time: 1.49s
Hidden count: 10    Parameters: 141      loss: 17.78656   Time: 1.47s
Hidden count: 15    Parameters: 286      loss: 17.98838   Time: 1.65s
Hidden count: 36    Parameters: 1441     loss: 16.80461   Time: 1.54s


## Activations

Different activation can have profound impact on model performance. Besides ```sigmoid```, which is just a different name for the logistic function, there are other activation function such as ```tanh``` and ```relu```. ```relu```, which stands for **RE**ctified **L**inear **U**nit, is a particular common choice due to its good performance.

In [20]:
polyDNN(data,hidden_count=1,layers=2,activation='relu')
polyDNN(data,hidden_count=10,layers=2,activation='relu')
polyDNN(data,hidden_count=15,layers=2,activation='relu')
polyDNN(data,hidden_count=36,layers=2,activation='relu')

Hidden count: 1     Parameters: 6        loss: 19.61141   Time: 1.36s
Hidden count: 10    Parameters: 141      loss: 14.55007   Time: 1.4s
Hidden count: 15    Parameters: 286      loss: 7.37173    Time: 1.43s
Hidden count: 36    Parameters: 1441     loss: 5.06284    Time: 1.45s
